In [1]:
!pip install nmrglue

  Using cached nmrglue-0.11-py2.py3-none-any.whl.metadata (3.3 kB)
  Using cached numpy-2.3.4-cp314-cp314-win_amd64.whl.metadata (60 kB)
  Using cached scipy-1.16.2-cp314-cp314-win_amd64.whl.metadata (60 kB)
Using cached nmrglue-0.11-py2.py3-none-any.whl (226 kB)
Using cached numpy-2.3.4-cp314-cp314-win_amd64.whl (12.9 MB)
Using cached scipy-1.16.2-cp314-cp314-win_amd64.whl (39.4 MB)

   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [numpy]
   ---------------------------------------- 0/3 [num

In [2]:
%pip install --upgrade pip setuptools wheel
%pip install ipykernel
%pip install numpy pandas matplotlib scipy nmrglue


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
Using cached setuptools-80.9.0-py3-none-any.whl (1.2 MB)
Using cached wheel-0.45.1-py3-none-any.whl (72 kB)

   ---------------------------------------- 0/2 [wheel]
   ---------------------------------------- 0/2 [wheel]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- ------------------- 1/2 [setuptools]
   -------------------- --------------

In [3]:
import sys
print("Kernel:", sys.executable)

import numpy as np, pandas as pd, matplotlib.pyplot as plt, nmrglue as ng
print("✅ Todo importó bien")


Kernel: c:\Users\fstraccia\Desktop\DL-NMR-main\.venv\Scripts\python.exe
✅ Todo importó bien


In [4]:
# Importacion de librerias

import numpy as np
import os
import nmrglue as ng
import random
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import re


In [ ]:
# direccion principal de los archivos MODIFICAR CON LA RUTA CORRESPONDIENTE

root_dir = r"C:\Users\fstraccia\Desktop\Simulacion RMN"


**Espectros individuales:**

Los espectros individuales puden ser obtenidos experimentalmente, por simulacion o por descarga en bases de datos. El siguiente ejemplo utiliza 7 espectros descargados desde HMDB que respresentan espectros de 7 compuestos obtenidos en un RMN con un campo=400Mz

In [7]:
# Carga de libreria de espectros individuales

# Listar los archivos
folder = os.path.join(root_dir, "HMDBSpectraFiles", "Espectros individuales")
filenames = os.listdir(folder)
filenames = filenames[:43]

# Cambiar directorio
os.chdir(folder)

# Cargar espectros
standardsSpec = []         # lista con los arrays de datos espectrales
standardsDictionary = []   # lista con los diccionarios de metadatos

for file in filenames:
    dic, data = ng.jcampdx.read(file)
    standardsSpec.append(data)
    standardsDictionary.append(dic)

**Carga de perfiles metabolicos:**

Un perfil metabolico es una archivo detallano los compuestos que se esperan mezclar, datos sobre los mismo (nombre, id hmbd, nombre INCHI, etc) y la distribucion estadistica de concentraciones que tendran en las mezclas simuladas.

In [8]:
%pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------

In [9]:
# CARGA DEL ARCHIVO QUE DEFINE PERFILES METABOLICOS
import os
excel_file = os.path.join(root_dir, "perfil metabolico.xlsx")
df = pd.read_excel(excel_file, sheet_name="Hoja1")

In [10]:
# Union de los espectros individuales con el perfil metabolico

def norm_name(s):
    s = str(s).strip()
    s = os.path.basename(s)
    return s.lower()

df["filename_norm"] = df["filename_jdx"].apply(norm_name)

# ) Construir el índice de los archivos cargados (desde `filenames`)

df_loaded = pd.DataFrame({
    "filename_disk": filenames,
    "idx": list(range(len(filenames)))
})
df_loaded["filename_norm"] = df_loaded["filename_disk"].apply(norm_name)

# === 3) Hacer el merge (match) ===
master_df = df.merge(
    df_loaded[["filename_norm", "idx", "filename_disk"]],
    on="filename_norm",
    how="left",
    validate="m:1"
)


**Creacion del eje:** Cada atomo de H que es puesto en el campo del RMN (wo) decae con una frecuencia especifica (w) que depende de su ambiente quimico. Los espectros reales se obtienen en el dominio del tiempo y se grafica la intensidad de cada nucleo no en termino de w sino en terminos de chemical shift (delta) que es la relacion de w de una w de referencia. Esto logra que independientemente del campo, el desplazamiento quimico de un atomo para una molecula dada es siempre constante.

Para poder graficar los datos de los espectros individuales subidos en formato jdx se debe crear un eje en ppm. Si se utilizasen espectros en FID, o se simulasen esto deberia modificarse acorde al tipo de archivo de espectros individuales utilizados.

In [11]:
# Creacion de un eje en ppm

def jdx_num(d, key, default=None, cast=float):
    v = d.get(key, default)
    if isinstance(v, list):
        v = v[0] if v else default
    try:
        return cast(v)
    except Exception:
        return default

d0 = standardsDictionary[0]

firstx = jdx_num(d0, 'FIRSTX', 10.0, float)
lastx  = jdx_num(d0, 'LASTX', 0.0, float)
npts   = jdx_num(d0, 'NPOINTS', len(standardsSpec[0]), int)

ppm_axis = np.linspace(firstx, lastx, npts)

# región de señal fija
SIG_START, SIG_END = 10000, 56000 #debe chequearse si estos puntos estan ok respecto a lo que se desea

# Chequeo de longitudes
assert all(len(s) == npts for s in standardsSpec), "Hay espectros con distinta longitud."

In [12]:
# Carpeta de salida
out_dir = os.path.join(root_dir, "Graficos")
os.makedirs(out_dir, exist_ok=True)

**Ploteo de los espectros individuales.**
En este ejemplo se grafican todos los espectros individuales cargados.En este caso, solo se cargaron 7 (muy poco para lo que se pretende en un futuro). Los espectros individuale se crean en la carpeta Graficos

In [14]:
# Ploteo de los espectros invidivuales
output_path = os.path.join(out_dir, "espectros_individuales.png")

# Carpeta de salida
out_dir = os.path.join(root_dir, "Graficos")
os.makedirs(out_dir, exist_ok=True)
output_path = os.path.join(out_dir, "espectros_individuales.png")

# ----------- GRÁFICO COMBINADO -----------
idxs = np.random.choice(len(standardsSpec), size=7, replace=False)
x = ppm_axis[SIG_START:SIG_END]

fig, ax = plt.subplots(figsize=(12, 4))

for i in idxs:
    y = standardsSpec[i][SIG_START:SIG_END]
    nombre_excel = master_df.loc[master_df["idx"] == i, "nombre"].iloc[0]
    ax.plot(x, y, label=f"{nombre_excel} (idx={i})")

ax.invert_xaxis()
ax.set_title("Espectros individuales")
ax.set_xlabel("ppm")
ax.set_ylabel("Intensidad")
ax.legend(fontsize=8)

fig.tight_layout()
fig.savefig(output_path, dpi=300, bbox_inches="tight")
plt.close(fig)   # <-- importante: cierra la figura sin mostrarla

print(f"✅ Gráfico guardado en: {output_path}")

#grafico los espectros individuales 
def safe_name(s: str, maxlen: int = 80) -> str:
    s = str(s).strip()
    s = re.sub(r"[^A-Za-z0-9._\- ]+", "_", s)   # reemplaza caracteres no válidos
    return s[:maxlen].rstrip(" ._-")
for i in idxs:
    y = standardsSpec[i][SIG_START:SIG_END]
    nombre_excel = master_df.loc[master_df["idx"] == i, "nombre"].iloc[0]
    fname = f"{safe_name(nombre_excel)}.png"          # nombre según Excel
    indiv_path = os.path.join(out_dir, fname)

    fig_i, ax_i = plt.subplots(figsize=(10, 3))
    ax_i.plot(x, y)
    ax_i.invert_xaxis()
    ax_i.set_title(f"{nombre_excel} (idx={i})")
    ax_i.set_xlabel("ppm")
    ax_i.set_ylabel("Intensidad")

    fig_i.tight_layout()
    fig_i.savefig(indiv_path, dpi=300, bbox_inches="tight")
    plt.close(fig_i)

    print(f"   ↳ guardado: {indiv_path}")

✅ Gráfico guardado en: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\espectros_individuales.png
   ↳ guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\7H-purin-6-amine.png
   ↳ guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\3-methyl-2-oxobutanoic acid.png
   ↳ guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\_3R_-3-hydroxybutanoic acid.png
   ↳ guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\_2R_3S_4R_5R_-5-_6-amino-9H-purin-9-yl_-3_4-dihydroxyoxolan-2-yl_methoxy_phospho.png
   ↳ guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\3-oxobutanoic acid.png
   ↳ guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\_2S_-2-hydroxybutanoic acid.png
   ↳ guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\1-Methylhistidine.png


**Agregado de estandar interno**

A la medicion de los espectros reales se le pueden agregar estandares internos. Estos son compuestos de referencia, de concentracion conocida. El objetivo es usar algunas de sus señales con algun fin en particular.

1- De cuantificacion: Como el desplazamiento quimico (delta en ppm) depende del nucleo, pero la intensidad depende solo de la cantidad de nucleos. Es posible cuantificar un compuesto relacionando el area de una señal de un compusto contra un area de una señal de otro compuesto de referencia de concentracion conocida.

2- De desplazamiento: Se pueden agregar estandares de desplazamiento para corregir cualquier error en la obtencion de los deltas.

3- De fase: Por errores de obtencion de la FID pueden existir errores de fase que son corregidos manualmente:  Ajustamos φ₀ hasta que el pico de referencia sea simétrico, sin hombros dispersivos. Luego ajustamos φ₁ para que los picos lejos del pivote también se vean bien. Volvémos a retocar φ₀ si hace falta (pequeñas iteraciones). Normalmente no se usan estandares de fase, pero un proyecto de IA puede requerirlo (talves)

In [19]:
#Lectura  Estandares internos

os.chdir(os.path.join(root_dir, "HMDBSpectraFiles"))
vd, v = ng.jcampdx.read("HMDB0000176_142710_predicted_H_400.jdx")  # Acido malico como estandar de cuantificacion

#se podria agregar TSP como estandar interno de desplazamiento (normalmente se usa TSP)

**A- Creacion de espectros mezclas poco realista:**

Creacion de un pipline para obtener espectros simulados a partir de espectros individuales perfectos. Se realiza por  síntesis de mezclas (iteraciones). Para cada mezcla se realiza una suma ponderada de todos los compuestos seleccionado a una concentracion definida en el perfil metabolico. Como los espectros individuales son obtenidos por descarga en HMDB son demaciado perfectos.

***1- Ausencia/presencia:*** Por compuesto según su p_ausencia segun probabilidad en
excel. Esto es solo para que no todos las  mezclas tengan los mismos compustos. Debatible respecto a los objetivos y tareas de las NN

***2- Concentración***:  muestreada según la distribución indicada en el Excel (Normal, Uniforme/Rectangular o Triangular) usando media y desvío.

***B- Creacion de espectros mezcla realistas:*** Todas las tecnicas de agregado de imperfecciones relistas debens ser analziadas para lograr mezclas simuladas realistas, incluso sin agregar inperfecciones muy poco frecuentes.

***1- Ruido***: Se agrega ruido sobre la linea de base, podrian existir mas de una forma de agregar ruido, tome una simple.

***2- Corrimiento aleatorio del espectro individual:*** Se hace que aleatorimente cada pico de un espectro individual se corra en puntos antes de mezclar con otros espectros individuales para producir la mezcla. Se hace asi porque el ejemplo del paper DL lo hacia de esa manera, se podria hacer talves de otra manera.

***Deformacion de picos***: ensanchamiento gaussiano/lorentziano (tipo Voigt) y, a veces, una cola asimétrica leve, porque este tipo la forma de los picos individuales rara vez existe en espectros reales. La manera de obetener estas deformaciones se realizo con chat gpt y con el recuerdo de un comentario de Raymundo, pero me falta entender bien como podriamos hacerlo.

***Desfase controlado***: Con una probabilidad prob, aplicamos fase 0 y 1 alrededor de un pivote ppm fijo (consistente en todo el dataset) usando la parte imaginaria generada por Hilbert. Guardamos las etiquetas phi0, phi1, ref_ppm y si se aplicó o no la fase. Esto tambien se hizo con bastante ayuda del chat gpt porque no se bien como se deberia aplicar esto. Seguramenre raymundo tiene mas ideas.



In [20]:
# Descriptores de concentraciones desde el perfil metabolico

#Función para decidir la inclusión de un compuesto en la mezcla según su probabilidad de ausencia
def entra_en_mezcla(p_ausencia):
    return np.random.rand() > float(p_ausencia)

# Función para muestrear la concentración de un compuesto según distribución estadística (Normal, Uniforme o Triangular
def sample_conc(media, desvio, distribucion):
    m, s = float(media), float(desvio)
    dist = str(distribucion).strip().upper()

    if dist.startswith("N"):
        return max(0.0, np.random.normal(m, s))

    if dist.startswith("R"):
        w = s * np.sqrt(12.0)
        a, b = max(0.0, m - w/2), max(m + w/2, m - w/2 + 1e-12)
        return np.random.uniform(a, b)

    if dist.startswith("T"):
        h = s * np.sqrt(6.0)
        a, c = max(0.0, m - h), max(m + h, m - h + 1e-12)
        return np.random.triangular(a, m, c)  # <-- antes decía np.triangular

    return m

In [21]:
# Función para aplicar un corrimiento aleatorio en puntos al espectro

max_shift = 30         # corrimiento máximo (en puntos)

def corrimiento_aleatorio(spec, max_shift):
    shift_pts = int(round(np.random.uniform(-1, 1) * max_shift))
    return ng.process.proc_base.cs(spec, shift_pts)


In [23]:
# Kernels para deformación de picos en el campo de la frecuencia. Se podria hacer desde el FID si los espectros infividuales estuviesen en FID o se puede volver a la FID desde los espectros.

#es necesario analizar muy bien este item, jugar con todos los parametros para generar deformaciones realistas y buscar otros script para generar estas deformaciones. Las deformaciones de picos no pueden ser resueltas por los sofwares de RMN durantes el procesamiento

def deformar_picos(spec,
                   gauss_sigma_range=(0.0, 2.0),   # ensanchamiento gaussiano en puntos
                   lorentz_gamma_range=(0.0, 2.0), # ensanchamiento lorentziano en puntos
                   asym_prob=0.2,                  # probabilidad de asimetría
                   asym_decay_pts=30,              # decaimiento de la cola (puntos)
                   asym_amp_range=(0.0, 0.10)):    # amplitud relativa de la cola
    y = spec

     #  Gaussiano
    sigma = np.random.uniform(*gauss_sigma_range)
    if sigma > 0:
        L = max(3, int(6*sigma)+1)
        x = np.arange(-(L//2), L//2 + 1)
        g = np.exp(-0.5*(x/sigma)**2)
        g /= g.sum()
        y = np.convolve(y, g, mode='same')

    #  Lorentziano
    gamma = np.random.uniform(*lorentz_gamma_range)
    if gamma > 0:
        L = max(3, int(10*gamma)+1)
        x = np.arange(-(L//2), L//2 + 1)
        l = 1.0 / (1.0 + (x/gamma)**2)
        l /= l.sum()
        y = np.convolve(y, l, mode='same')

    #  Asimetría
    if np.random.rand() < asym_prob and asym_decay_pts > 0:
        amp = np.random.uniform(*asym_amp_range)
        dir_sign = np.random.choice([-1, 1])
        L = max(3, int(5*asym_decay_pts))
        x = np.arange(0, L)
        e = np.exp(-x / float(asym_decay_pts))
        e /= e.sum()

        # núcleo: delta + pequeña cola a un lado
        k = np.zeros(2*L+1)
        k[L] = 1.0 - amp
        if dir_sign > 0:
            k[L+1:L+1+L] = amp * e               # cola a la derecha
        else:
            k[L - L:L] = (amp * e)[::-1]         # cola a la izquierda (equivale a k[:L] = (amp*e)[::-1])

        y = np.convolve(y, k, mode='same')

    return y


In [24]:
def aplicar_fase(y, x_ppm, ref_ppm=0.0, prob=1,
                 phi0_range=(-15, 15),     # grados
                 phi1_range=(-30, 30)):    # grados/ppm

    aplicar = (np.random.rand() < prob)

    # Parámetros de fase
    phi0 = np.random.uniform(*phi0_range) if aplicar else 0.0
    phi1 = np.random.uniform(*phi1_range) if aplicar else 0.0

    # Parte analítica y rotación
    A = np.real(hilbert(y))
    D = np.imag(hilbert(y))
    phi = (phi0 + phi1 * (x_ppm - ref_ppm)) * (np.pi / 180.0)
    y_out = A * np.cos(phi) - D * np.sin(phi)

    return y_out, int(aplicar), float(phi0), float(phi1), float(ref_ppm)

In [25]:
from scipy.signal import hilbert

#  Generacion de espectros por mezcla de compuestos individuales: La mezcla contiene kernels de deformacion de picos, y corrimiento en espectro individual anterior a la mezcla y un corrimiento de fase

np.random.seed(2)
random.seed(2)

iterations = 10000      # cuántas mezclas sintéticas generar
standard_scale = 1.0    # factor de intensidad fijo para el estándar

spectra = []
concs = []  # lista paralela para guardar las concentraciones por mezcla

REF_PPM = float(np.median(ppm_axis))

phase_applied_list, phi0_list, phi1_list, ref_list = [], [], [], []

# ── Bucle de mezclas ───────────────────────────────────────────────────────────
for _ in range(iterations):
    mix = np.zeros_like(standardsSpec[0])
    conc_dict = {}

    # sumar compuestos con ausencia, concentración y corrimiento/deformación
    for _, row in master_df.iterrows():
        if not entra_en_mezcla(row["p_ausencia"]):
            continue
        conc = sample_conc(row["media"], row["desvio"], row["distribucion"])
        spec = standardsSpec[int(row["idx"])]
        spec_shifted  = corrimiento_aleatorio(spec, max_shift)
        spec_deformed = deformar_picos(spec_shifted)
        mix += conc * spec_deformed
        conc_dict[int(row["idx"])] = float(conc)

    # agregar estándar interno
    m = min(len(mix), len(v))
    mix = mix[:m] + standard_scale * v[:m]
    conc_dict["_IS"] = float(standard_scale)

    # aplicar fase con probabilidad prob y pivote fijo (3)
    mix, applied, phi0_used, phi1_used, ref_used = aplicar_fase(
        mix, ppm_axis[:len(mix)], ref_ppm=REF_PPM, prob=0.7
    )
    phase_applied_list.append(int(applied))
    phi0_list.append(phi0_used)
    phi1_list.append(phi1_used)
    ref_list.append(ref_used)

    # baseline y ruido
    baseline_offset = np.random.uniform(-0.1, 0.1)
    mix = mix + baseline_offset

    noise_sigma = np.random.uniform(0.01, 0.1)
    mix = mix + np.random.normal(0, noise_sigma, size=mix.shape)

    # guardar
    spectra.append(mix)
    concs.append(conc_dict)


**Visualizaciones**:

Se grafica aleatoriamente algun espectro, con su zoom para ver como corrio el script.Tambien pedido a chat gpt, seguro hay formas mas limpias de hacerlo

Nota: HAY ALGUN ERROR EN LA CREACION DEL EJE PPM PORQUE LOS DATOS DE CSV DE CADA MEZCLA SALEN CON UN EJE 12 a -2 CUANDO DEBIESE SER 10 a 0

In [26]:
# --- config de salida ---
out_dir = os.path.join(root_dir, "Graficos")
os.makedirs(out_dir, exist_ok=True)

# --- función utilitaria ---
def cut_ppm(x_axis, y, ppm_min, ppm_max):
    i0 = int(np.argmin(np.abs(x_axis - ppm_min)))
    i1 = int(np.argmin(np.abs(x_axis - ppm_max)))
    lo, hi = sorted([i0, i1])
    return x_axis[lo:hi], y[lo:hi]

# ventanas de zoom (ppm)
zoom_windows = [(0.0, 2.5), (2.5, 3.5), (3.5, 5.0), (5.0, 7.5), (7.5, 10.0)]

# elegir 4 mezclas aleatorias (sin reemplazo)
mix_idxs = np.random.choice(len(spectra), size=4, replace=False)

# eje para la ventana señal (el que ya usabas)
x_full = ppm_axis[SIG_START:SIG_END]

for k in mix_idxs:
    # carpeta por mezcla
    mix_dir = os.path.join(out_dir, f"mix_{int(k)}")
    os.makedirs(mix_dir, exist_ok=True)

    # ---------- espectro completo ----------
    y_full = spectra[k][SIG_START:SIG_END]

    fig, ax = plt.subplots(figsize=(12, 3))
    ax.plot(x_full, y_full)
    ax.invert_xaxis()
    ax.set_title(f"Mezcla sintética #{k} (completo)")
    ax.set_xlabel("ppm"); ax.set_ylabel("Intensidad")
    fig.tight_layout()

    full_path = os.path.join(mix_dir, f"mix_{int(k)}_completo.png")
    fig.savefig(full_path, dpi=300, bbox_inches="tight")
    plt.close(fig)

    # ---------- zooms ----------
    y_all = spectra[k]  # para cortar por ppm exacta sobre el eje completo
    for (zmin, zmax) in zoom_windows:
        xz, yz = cut_ppm(ppm_axis, y_all, zmin, zmax)

        fig, ax = plt.subplots(figsize=(12, 3))
        ax.plot(xz, yz)
        ax.invert_xaxis()
        ax.set_title(f"Mezcla #{k} — zoom {zmin}–{zmax} ppm")
        ax.set_xlabel("ppm"); ax.set_ylabel("Intensidad")
        fig.tight_layout()

        # nombre de archivo legible
        zname = f"{zmin:.1f}-{zmax:.1f}".replace(".", "p")
        zoom_path = os.path.join(mix_dir, f"mix_{int(k)}_zoom_{zname}_ppm.png")
        fig.savefig(zoom_path, dpi=300, bbox_inches="tight")
        plt.close(fig)

    print(f"✅ Guardado: {mix_dir}")


✅ Guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\mix_2762
✅ Guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\mix_9846
✅ Guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\mix_7259
✅ Guardado: C:\Users\fstraccia\Desktop\DL-NMR-main\Graficos\mix_6676


**Obtencion de archivos CSV de las mezclas y un excel con sus metadatos:** En esta seccion se deben juntar todos los archivos necesarios y con la informacion requerida, para ser utilizados como alimentacion de redes neuronales. El archivo de metadatos deberia mejorarse en cada instancia y objetivos de creacion de una NN

In [ ]:
# Carpeta de salida directamente en root_dir
export_dir = os.path.join(root_dir, "Espectros sinteticos")

# Crear carpeta si no existe (solo por seguridad)
os.makedirs(export_dir, exist_ok=True)

# Detectar cantidad de mezclas (lista vs array)
if isinstance(spectra, (list, tuple)):
    N = len(spectra)
    def get_mix(i): return np.asarray(spectra[i])
else:
    N = spectra.shape[0]
    def get_mix(i): return np.asarray(spectra[i, :])

print("Mezclas a exportar:", N)

def safe_name(s):
    s = str(s).strip()
    return "".join(c if c.isalnum() or c in " ._-" else "_" for c in s)

num_saved = 0
for i in range(N):
    y = get_mix(i)
    x = ppm_axis[:len(y)]
    if 'mix_names' in globals() and i < len(mix_names):
        base = safe_name(mix_names[i])
    else:
        base = f"Mezcla_{i:03d}"

    fname = os.path.join(export_dir, f"{base}.csv")

    # Evitar sobreescritura
    if os.path.exists(fname):
        fname = os.path.join(export_dir, f"{base}_{i:03d}.csv")

    pd.DataFrame({"ppm": x, "intensity": y}).to_csv(fname, index=False)
    num_saved += 1

print(f"✅ {num_saved} CSV guardados en: {export_dir}")

# Listar algunos para verificar
ejemplos = sorted([f for f in os.listdir(export_dir) if f.endswith('.csv')])[:5]
print("Ejemplos:", ejemplos)


Mezclas a exportar: 10000


In [ ]:
# Carpeta de salida en Drive

export_dir = os.path.join(root_dir, "Espectros mezcla")
os.makedirs(export_dir, exist_ok=True)

# Detectar cantidad de mezclas (lista vs array)
if isinstance(spectra, (list, tuple)):
    N = len(spectra)
    def get_mix(i): return np.asarray(spectra[i])
else:
    # asume array (N, L)
    N = spectra.shape[0]
    def get_mix(i): return np.asarray(spectra[i, :])

print("Mezclas a exportar:", N)

def safe_name(s):
    s = str(s).strip()
    return "".join(c if c.isalnum() or c in " ._-" else "_" for c in s)

num_saved = 0
for i in range(N):
    y = get_mix(i)
    x = ppm_axis[:len(y)]
    # Nombre por mezcla (usa mix_names si existe)
    if 'mix_names' in globals() and i < len(mix_names):
        base = safe_name(mix_names[i])
    else:
        base = f"Mezcla {i:03d}"
    fname = os.path.join(export_dir, f"{base}.csv")
    # Evitar pisar si ya existe
    if os.path.exists(fname):
        fname = os.path.join(export_dir, f"{base}_{i:03d}.csv")

    pd.DataFrame({"ppm": x, "intensity": y}).to_csv(fname, index=False)
    num_saved += 1

print(f"OK: {num_saved} CSV guardados en: {export_dir}")

# Listar algunos para verificar
ejemplos = sorted([f for f in os.listdir(export_dir) if f.endswith(".csv")])[:5]
print("Ejemplos:", ejemplos)

In [ ]:
!pip install -q XlsxWriter


# 1) Orden maestro (por idx) para que las columnas queden consistentes
mdf_idx = (
    master_df[['idx','nombre']]
    .dropna(subset=['idx'])
    .drop_duplicates(subset=['idx'])
    .copy()
)
mdf_idx['idx'] = mdf_idx['idx'].astype(int)

order_idx = mdf_idx.sort_values('idx')['idx'].tolist()
metabolite_names = mdf_idx.sort_values('idx')['nombre'].tolist()

# 2) Construir matriz Y (N mezclas x M metabolitos) desde 'concs'
N = len(concs)
M = len(order_idx)
idx2col = {idx:j for j, idx in enumerate(order_idx)}

y_conc = np.zeros((N, M), dtype=float)
is_vec = np.zeros((N, 1), dtype=float)  # estándar interno

for i, d in enumerate(concs):           # d: dict de concentraciones de esa mezcla
    # metabolitos
    for k, val in d.items():
        if k == "_IS":
            continue
        j = idx2col.get(int(k))
        if j is not None:
            y_conc[i, j] = float(val)
    # estándar interno (si lo guardaste como "_IS")
    is_vec[i, 0] = float(d.get("_IS", 0.0))

standard_name = "Maleic acid (IS)"  # cambiá el nombre si querés

# 3) DataFrame final (mix_id + concentraciones + IS)
dfY = pd.DataFrame(y_conc, columns=metabolite_names)
dfY[standard_name] = np.asarray(is_vec).ravel()
dfY.insert(0, "mix_id", np.arange(N))

dfY["phase_applied"]   = phase_applied_list
dfY["phi0_deg"]        = phi0_list
dfY["phi1_deg_per_ppm"]= phi1_list
dfY["ref_ppm"]         = ref_list

cols = (["mix_id","phase_applied","phi0_deg","phi1_deg_per_ppm","ref_ppm",
         standard_name] + metabolite_names)
dfY = dfY[cols]

# 4) Guardar en Excel en /DL-NMR-main/Data set/mezclas_concentraciones.xlsx
export_dir = os.path.join(root_dir, "Data set")
os.makedirs(export_dir, exist_ok=True)

xlsx_path = os.path.join(export_dir, "mezclas_concentraciones.xlsx")

with pd.ExcelWriter(xlsx_path, engine="xlsxwriter") as writer:
    dfY.to_excel(writer, sheet_name="concentraciones", index=False)

print(f"✅ Archivo Excel guardado en: {xlsx_path} | shape: {dfY.shape}")

